In [9]:
import util
import numpy as np
import math
import matplotlib.pyplot as plt
# !pip install chart_studio
from scipy.signal import medfilt
from plotly import tools
import chart_studio.plotly as py
import plotly.graph_objs as go
import pose
from pose_parser import parse_file

In [11]:
ps = parse_file("dataset/front/front_bicep_8.npy")

Data shape:  (1069, 19, 3)


TypeError: __init__() missing 7 required positional arguments: 'LBigToe', 'LSmallToe', 'LHeel', 'RBigToe', 'RSmallToe', 'RHeel', and 'Background'

In [60]:
frames = ps
left_upperarm_forearm_angles = []
right_upperarm_forearm_angles = []
left_refer_angles = []
right_refer_angles = []
dire = ["Upward"]

In [61]:
for i,frame in enumerate(frames):
    if (i==0):
        initial_position_left = pose.Part(frame.lelbow, frame.lwrist)
        initial_position_right = pose.Part(frame.relbow,frame.rwrist)

    right_upperarm = pose.Part(frame.relbow, frame.rshoulder)
    right_forearm = pose.Part(frame.relbow, frame.rwrist)
    left_upperarm = pose.Part(frame.lelbow, frame.lshoulder)
    left_forearm = pose.Part(frame.lelbow, frame.lwrist)

    left_refer_angle = initial_position_left.calculate_angle(left_forearm)
    right_refer_angle = initial_position_right.calculate_angle(right_forearm)

    left_angle = left_upperarm.calculate_angle(left_forearm)
    right_angle = right_upperarm.calculate_angle(right_forearm)

        
    left_refer_angles.append(left_refer_angle)
    right_refer_angles.append(right_refer_angle)

    left_upperarm_forearm_angles.append(left_angle)
    right_upperarm_forearm_angles.append(right_angle)


In [62]:
left_refer_angles = medfilt (left_refer_angles, 31)
right_refer_angles = medfilt (right_refer_angles, 31)
    
left_upperarm_forearm_angles= medfilt(left_upperarm_forearm_angles, 31)
right_upperarm_forearm_angles = medfilt(right_upperarm_forearm_angles,31)

for i in range(1, len(left_refer_angles)):
    prev_angle = left_refer_angles [i-1]
    current_angle = left_refer_angles [i]

    if(current_angle - prev_angle > 0):
        dire.append("Upward")
    elif (current_angle- prev_angle <0):
        dire.append("Downward")
    else:
        dire.append("Stationary")

feedback = ''
for i in range(1, len(dire)):
    if (dire[i-1]!=dire[i] and dire[i]!=dire[i+1] and dire [i-1]== dire[i+1]):
        dire[i]=dire[i+1]
    if (dire[i] == 'Stationary' and dire[i+1]== "Upward"):
        if(left_upperarm_forearm_angles[i]<170.0):
            print("Stretch your arm all the way to bottom")
    if (dire[i] == 'Stationary' and dire[i+1]== "Downward"):
        if (left_upperarm_forearm_angles[i] > 30.0 and left_upperarm_forearm_angles[i]<160.0):
            print("Squeeze your arm all the way to top properly")


Squeeze your arm all the way to top properly
Squeeze your arm all the way to top properly
Squeeze your arm all the way to top properly
Stretch your arm all the way to bottom
Squeeze your arm all the way to top properly
Squeeze your arm all the way to top properly
Squeeze your arm all the way to top properly
Squeeze your arm all the way to top properly
Stretch your arm all the way to bottom
Squeeze your arm all the way to top properly


In [63]:
import cv2
import numpy as np
import time

In [64]:
cap = cv2.VideoCapture('videos/bicep_8.mp4')
ps2 = parse_file("dataset/front/front_bicep_8.npy", normalize=False)
if(cap.isOpened()==False):
    print("Error")
i= 0
reps = 0
feedback = ''
while(cap.isOpened()):
    ret, frame = cap.read()
    if (ret== True):
        
        for name, joint in ps2[i]:
            x = int(joint.x)
            y = int(joint.y)
            cv2.circle(frame, (x, y), 1, (0, 255, 255), 2)
        
        
        if (dire[i] == 'Stationary' and dire[i+1]== "Upward"):
            reps+=1
            if(left_upperarm_forearm_angles[i]<170.0):
                feedback+= "Stretch your arm all the way to bottom"
        if (dire[i] == 'Stationary' and dire[i+1]== "Downward"):
            if (left_upperarm_forearm_angles[i] > 30.0 and left_upperarm_forearm_angles[i]<160.0):
                feedback+="Squeeze your arm all the way to top properly"
        text = str(round(left_upperarm_forearm_angles[i],2))+ "  " + str(round(right_upperarm_forearm_angles[i],2)) + " " + dire[i] + "Frame:" + str(i)
        cv2.putText(frame, text, (10,50),cv2.FONT_HERSHEY_COMPLEX,0.6,(0,255,255),2)

        text = "Reps: {}".format(reps)
        cv2.putText(frame, text,(10, 100),cv2.FONT_HERSHEY_COMPLEX,0.6,(0,255,255),2 )
        cv2.imshow('Frame', frame)
        i=i+1
        if (cv2.waitKey(25) & 0xFF == ord('q')):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()

Data shape:  (820, 19, 3)
